In [2]:
from IPython.display import display, Markdown

# Update the notebook title and descriptions to reflect a general philosophy recommender


# Update the main title and intro
display(Markdown("""
# Philosophy Recommender RAG System

This notebook implements a Retrieval Augmented Generation (RAG) system that provides philosophical advice based on a variety of philosophical teachings. It uses:
- A PDF document containing philosophical texts
- LangChain for RAG implementation
- OpenAI's GPT model for generating compassionate, contextual responses
- Vector storage for semantic search

The system will:
1. Load and process philosophical teachings from PDF
2. Create embeddings for semantic search
3. Take user queries about emotional challenges
4. Provide thoughtful advice grounded in philosophy
"""))

# Update section 2 markdown
display(Markdown("""
## 2. Load and Process Text Document

We'll now:
1. Load the philosophical teachings from a text file
2. Split it into manageable chunks
3. Create Document objects for the RAG system

The text splitter is configured to preserve context while creating chunks that are small enough for effective retrieval.
"""))

# Update section 3 markdown
display(Markdown("""
## 3. Create Vector Store

Now we'll:
1. Initialize the embeddings model
2. Create embeddings for all chunks
3. Store them in a vector store for similarity search

We'll use OpenAI's embedding model for high-quality semantic search.
"""))

# Update section 4 markdown
display(Markdown("""
## 4. Create Response Generator

We'll create a chain that:
1. Takes a user's emotional concern
2. Retrieves relevant philosophical wisdom
3. Generates a compassionate, thoughtful response

The prompt is designed to:
- Show empathy for the user's feelings
- Ground advice in philosophical teachings
- Offer practical suggestions
- Maintain a warm, supportive tone
"""))

# Update section 5 markdown
display(Markdown("""
## 5. Interactive Guidance System

Now you can use the cell below to ask for guidance about any emotional concern. The system will:
1. Find relevant philosophical teachings
2. Generate thoughtful, compassionate advice
3. Provide practical suggestions based on philosophical wisdom

Try asking about feelings like:
- Anxiety
- Stress
- Uncertainty
- Loneliness
- Frustration
"""))


# Philosophy Recommender RAG System

This notebook implements a Retrieval Augmented Generation (RAG) system that provides philosophical advice based on a variety of philosophical teachings. It uses:
- A PDF document containing philosophical texts
- LangChain for RAG implementation
- OpenAI's GPT model for generating compassionate, contextual responses
- Vector storage for semantic search

The system will:
1. Load and process philosophical teachings from PDF
2. Create embeddings for semantic search
3. Take user queries about emotional challenges
4. Provide thoughtful advice grounded in philosophy



## 2. Load and Process Text Document

We'll now:
1. Load the philosophical teachings from a text file
2. Split it into manageable chunks
3. Create Document objects for the RAG system

The text splitter is configured to preserve context while creating chunks that are small enough for effective retrieval.



## 3. Create Vector Store

Now we'll:
1. Initialize the embeddings model
2. Create embeddings for all chunks
3. Store them in a vector store for similarity search

We'll use OpenAI's embedding model for high-quality semantic search.



## 4. Create Response Generator

We'll create a chain that:
1. Takes a user's emotional concern
2. Retrieves relevant philosophical wisdom
3. Generates a compassionate, thoughtful response

The prompt is designed to:
- Show empathy for the user's feelings
- Ground advice in philosophical teachings
- Offer practical suggestions
- Maintain a warm, supportive tone



## 5. Interactive Guidance System

Now you can use the cell below to ask for guidance about any emotional concern. The system will:
1. Find relevant philosophical teachings
2. Generate thoughtful, compassionate advice
3. Provide practical suggestions based on philosophical wisdom

Try asking about feelings like:
- Anxiety
- Stress
- Uncertainty
- Loneliness
- Frustration


## 1. Setup and Dependencies

First, we'll import the required libraries:
- `PyPDF2` for reading PDF files
- `langchain` components for RAG
- `dotenv` for environment variables
- Custom prompt templates for compassionate responses

In [2]:
from __future__ import annotations

import os
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore

# Load environment variables
load_dotenv()

# Configure API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "OPENROUTER_API_KEY not found in .env file"
    )

## 2. Load and Process Text Document

We'll now:
1. Load the Buddhist teachings from a text file
2. Split it into manageable chunks
3. Create Document objects for the RAG system

The text splitter is configured to preserve context while creating chunks that are small enough for effective retrieval.

In [3]:
# Load text file
import glob
from langchain.document_loaders import PyPDFLoader

# Find all PDF files in the current directory (or specify your path)
pdf_files = glob.glob("*.pdf")

# Load all PDFs and concatenate their documents
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)

documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs")

# loader = PyPDFLoader("stoicism.pdf")
# documents = loader.load()

print('Document:', documents)
# Create initial document
# documents = [Document(page_content=text_content)]

# Configure text splitter for optimal chunk size
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)

# Split documents into chunks
chunks = text_splitter.split_documents(documents)
print(f"Split text into {len(chunks)} chunks")

# Preview first chunk
if chunks:
    print("\nSample chunk content:")
    print("=" * 40)
    print(chunks[0].page_content[:200], "...")

incorrect startxref pointer(3)
parsing for Object Streams
parsing for Object Streams


Loaded 1 documents from 1 PDFs
Document: [Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': 'stoicism.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Stoicism: Ancient Wisdom for Modern Resilience\nStoicism is a practical philosophy that emphasizes personal ethics,\nrational thinking, and emotional resilience. Founded in ancient Greece\nand later flourishing in Rome, Stoicism provides timeless wisdom for\ndealing with life's challenges.\nCore Principles of Stoicism:\n1. The Dichotomy of Control\n- Focus only on what you can control\n- Accept what you cannot change\n- External events are neutral; our judgments create our experience\n- Channel energy only into actionable matters\n2. Virtue as the Highest Good\n- Wisdom: Clear judgment and rational thinking\n- Justice: Fairness and service to others\n- Courage: Facing challenges with fortitude\n- Temperance: Self-control and moderation\n3. Living According to Nature\n- Accept re

## 3. Create Vector Store

Now we'll:
1. Initialize the embeddings model
2. Create embeddings for all chunks
3. Store them in a vector store for similarity search

We'll use OpenAI's embedding model for high-quality semantic search.

In [4]:
# Initialize the embedding model
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # Use a standard embedding model name
    openai_api_key=api_key,
    openai_api_base="https://api.openai.com/v1",
)

# Create embeddings and store them in a vector store
vectorstore = InMemoryVectorStore.from_documents(
    chunks,
    embeddings_model
)

## 4. Create Response Generator

We'll create a chain that:
1. Takes a user's emotional concern
2. Retrieves relevant Buddhist wisdom
3. Generates a compassionate, thoughtful response

The prompt is designed to:
- Show empathy for the user's feelings
- Ground advice in Buddhist teachings
- Offer practical suggestions
- Maintain a warm, supportive tone

In [5]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-5-mini",  # or another suitable model
)

# Create a thoughtful prompt template
template = """You are a thoughtful philosophy recommender chatbot focused on mental wellness (not medical advice). 
Given the user's concern and the philosophical teachings provided below, recommend the philosophy or school of thought 
that best matches their situation. Offer a brief explanation of why this philosophy is relevant, share a key idea or quote, 
and suggest practical next steps inspired by this philosophy. 

Context from philosophical teachings:
{context}

User's concern: {question}

Your response should:
1. Show empathy and understanding for the user's feelings
2. Clearly state which philosophy or tradition is most relevant and why
3. Share a key idea, principle, or quote from that philosophy
4. Suggest practical, actionable steps the user can take based on this philosophy
5. Remind the user this is for mental wellness and not a substitute for medical advice

Response:"""

prompt = ChatPromptTemplate.from_template(template)

def get_buddhist_guidance(question: str, num_chunks: int = 3) -> str:
    """Generate Buddhist-inspired guidance for an emotional concern."""
    # Retrieve relevant passages
    context_chunks = vectorstore.similarity_search(question, k=num_chunks)
    context = "\n\n".join(doc.page_content for doc in context_chunks)
    
    # Generate response
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context": context, "question": question})
    return response


## 5. Interactive Guidance System

Now you can use the cell below to ask for guidance about any emotional concern. The system will:
1. Find relevant Buddhist teachings
2. Generate thoughtful, compassionate advice
3. Provide practical suggestions based on Buddhist wisdom

Try asking about feelings like:
- Anxiety
- Stress
- Uncertainty
- Loneliness
- Frustration

In [7]:
# Interactive cell for seeking guidance
question = "I feel like nothing is in my control, I am putting all efforts towards my goals, but nothing is working out for me."  # Replace with your concern
print("Your concern:", question)
print("\nPhilosophical advice:")
print("=" * 60)
print(get_buddhist_guidance(question))

Your concern: I feel like nothing is in my control, I am putting all efforts towards my goals, but nothing is working out for me.

Philosophical advice:
I’m really sorry you’re feeling that way — feeling like nothing is in your control despite trying hard is exhausting and demoralizing. That experience is understandable, and it’s okay to want a practical way to respond.

Most relevant philosophy: Stoicism
Why: Stoicism directly addresses the exact struggle you describe. Its central teaching — the Dichotomy of Control — helps you separate what you can influence from what you can’t, and it trains your attention and energy toward effective, reasoned action rather than toward frustration about outcomes. Stoicism also offers concrete practices for steadying emotion and rebuilding agency.

Key idea / quote
- The Dichotomy of Control: focus on what you can control; accept what you cannot.
- Marcus Aurelius: “You have power over your mind — not outside events. Realize this, and you will find s